In [1]:
from tensorflow import keras
import numpy as np
from matplotlib import pyplot as plt
import nrrd
from pathlib import Path

## Exploring prediction of biomarkers with CNN's

### T1 
T1 sequences appear to have a high predictive power through the use of radiomics features. Are these biomarkers also predictable via the use of a covnet. Which areas of the kidney contribut the most to these predictions? We can create a dataset using the unlabeled images and the encoded ischemia status.

In [2]:
files = Path("../sequences/Dataset001_T1/imagesRd/").glob("*")
all_images = []
all_labels = []
for file in files:
    all_labels += [file.name.split("_")[-2]] * 24
    all_images.append(nrrd.read(file)[0])
all_images = np.array(all_images)
all_images.shape

(126, 192, 192, 24)

#### formatting the array in a format, tensorflow can accept

In [3]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

all_labels = le.fit_transform(all_labels)

In [4]:
images = all_images.reshape((-1, 192, 192, 1))
labels = np.array(all_labels)

print(f"Dimensions of Images: {images.shape}\nLabels: {labels.shape}")

Dimensions of Images: (3024, 192, 192, 1)
Labels: (3024,)


#### Creating a train-test-validation split

In [5]:
from sklearn.model_selection import train_test_split

itrain, itest, ltrain, ltest = train_test_split(images, labels, train_size=.8)
itrain.shape, itest.shape, ltrain.shape, ltest.shape

((2419, 192, 192, 1), (605, 192, 192, 1), (2419,), (605,))

## Assemble CCNetwork

In [8]:
model = keras.models.Sequential()

# Input layer
model.add(keras.layers.Input(shape=(192, 192, 1,)))
model.add(keras.layers.Rescaling(scale=1/images.max()))


# Deep layers 1 
model.add(keras.layers.Conv2D(128, kernel_size=(3,3), activation="relu"))
model.add(keras.layers.MaxPooling2D(pool_size=2))

# Deep layers 2 
model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation="relu"))
model.add(keras.layers.MaxPooling2D(pool_size=2))

# # Deep layers 3
# model.add(keras.layers.Conv2D(16, kernel_size=(3,3), activation="relu"))
# model.add(keras.layers.MaxPooling2D(pool_size=2))

# Convulsion layers
model.add(keras.layers.Flatten())
# model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(1, activation=keras.activations.relu))

# summary
model.summary() 

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ (None, 192, 192, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 190, 190, 128)  │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 95, 95, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 93, 93, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 46, 46, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 135424)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │       135,425 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 210,497 (822.25 KB)

 Trainable params: 210,497 (822.25 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.compile(
    optimizer = "adam",
    loss = "crossentropy",
    metrics=["accuracy"]
)
history = model.fit(
    x = itrain,
    y = ltrain,
    epochs=10,
    batch_size=200,
    validation_data=(itest, ltest)
)

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 150s 11s/step - accuracy: 0.4894 - loss: 8.2306 - val_accuracy: 0.5157 - val_loss: 7.8060
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 136s 10s/step - accuracy: 0.4900 - loss: 8.2205 - val_accuracy: 0.5157 - val_loss: 7.8060
Epoch 3/10
 7/13 ━━━━━━━━━━━━━━━━━━━━ 1:48 18s/step - accuracy: 0.4929 - loss: 8.1739

KeyboardInterrupt: 

In [10]:
# Input image dimensions (128x128 grayscale)
from keras import layers

input_shape = (192, 192, 1)

def build_deep_tissue_classifier(input_shape, num_classes):
    model = keras.Sequential([
        # Block 1
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25), # Dropout after pooling

        # Block 2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Block 3
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Flatten and Dense layers for classification
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5), # More aggressive dropout before the final classification layer
        layers.Dense(num_classes, activation='softmax') # Softmax for multi-class classification
    ])
    return model

# Create the model
model = build_deep_tissue_classifier(input_shape, 1)

# Compile the model
# Using Adam optimizer with a default learning rate (you might tune this)
# Loss function depends on your label encoding (categorical_crossentropy for one-hot)
# Metrics to monitor during training
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='crossentropy', # Use 'sparse_categorical_crossentropy' if labels are integer encoded
              metrics=['accuracy'])

In [11]:
history = model.fit(
    x = itrain,
    y = ltrain,
    epochs=10,
    batch_size=32,
    validation_data=(itest, ltest)
)

Epoch 1/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 1062s 14s/step - accuracy: 0.4951 - loss: 1.1351 - val_accuracy: 0.5008 - val_loss: 1.0161
Epoch 2/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 193s 3s/step - accuracy: 0.5086 - loss: 0.7155 - val_accuracy: 0.5008 - val_loss: 0.7261
Epoch 3/10
38/76 ━━━━━━━━━━━━━━━━━━━━ 1:39 3s/step - accuracy: 0.4765 - loss: 0.6866

KeyboardInterrupt: 